# Análisis Exploratorio de Datos de Emisiones

Este notebook contiene el análisis exploratorio de datos (EDA) y análisis estadístico de las emisiones, utilizando Polars, NumPy y SciPy.

In [ ]:
import polars as pl
import numpy as np
from scipy import stats

# Cargar el DataFrame procesado (asumiendo que data_processing.py lo guarda o lo devuelve)
# Para este ejemplo, cargaremos directamente el CSV y aplicaremos el preprocesamiento básico aquí
# En un escenario real, se importaría la función process_data y se usaría el DataFrame resultante.

def process_data_for_notebook(file_path):
    df = pl.read_csv(file_path)
    df = df.rename({
        'AÑO': 'Anio',
        'CLASIFICACION': 'Clasificacion',
        'CH4_eq': 'CH4_eq',
        'CO2_eq': 'CO2_eq',
        'N2O_eq': 'N2O_eq',
        'Total_Emisiones': 'Total_Emisiones',
        'Emisiones_netas': 'Emisiones_netas'
    })
    numeric_cols = ['CH4_eq', 'CO2_eq', 'N2O_eq', 'Total_Emisiones', 'Emisiones_netas']
    for col in numeric_cols:
        if col in df.columns:
            df = df.with_columns(pl.col(col).cast(pl.String).str.replace_all(",", ".").cast(pl.Float64))
    for col in numeric_cols:
        if col in df.columns:
            df = df.with_columns(pl.col(col).fill_null(0))
    return df

file_path = '/home/ubuntu/upload/proyecto2.csv'
df = process_data_for_notebook(file_path)

print("DataFrame cargado y preprocesado:")
print(df.head())

## Estadísticas Descriptivas

Vamos a obtener un resumen estadístico de las columnas numéricas.

In [ ]:
print(df.describe())

## Clasificaciones Únicas

Exploramos las diferentes clasificaciones de emisiones presentes en el dataset.

In [ ]:
print("Clasificaciones únicas:")
print(df['Clasificacion'].unique())

## Emisiones Totales a lo largo del Tiempo

Analizamos la suma total de emisiones por año.

In [ ]:
emisiones_por_anio = df.group_by('Anio').agg(pl.sum('Total_Emisiones').alias('Total_Emisiones_Anual')).sort('Anio')
print(emisiones_por_anio)

## Análisis de Correlación con SciPy

Calculamos la correlación entre los diferentes tipos de gases de efecto invernadero (GEI).

In [ ]:
ch4_emissions = df['CH4_eq'].to_numpy()
co2_emissions = df['CO2_eq'].to_numpy()
n2o_emissions = df['N2O_eq'].to_numpy()

# Eliminar NaN si los hubiera (aunque ya se rellenaron con 0 en el preprocesamiento)
ch4_emissions = ch4_emissions[~np.isnan(ch4_emissions)]
co2_emissions = co2_emissions[~np.isnan(co2_emissions)]
n2o_emissions = n2o_emissions[~np.isnan(n2o_emissions)]

# Asegurarse de que todos los arrays tienen la misma longitud para la correlación
min_len = min(len(ch4_emissions), len(co2_emissions), len(n2o_emissions))
ch4_emissions = ch4_emissions[:min_len]
co2_emissions = co2_emissions[:min_len]
n2o_emissions = n2o_emissions[:min_len]

if min_len > 1:
    corr_ch4_co2, _ = stats.pearsonr(ch4_emissions, co2_emissions)
    corr_ch4_n2o, _ = stats.pearsonr(ch4_emissions, n2o_emissions)
    corr_co2_n2o, _ = stats.pearsonr(co2_emissions, n2o_emissions)
    print(f"Correlación Pearson entre CH4_eq y CO2_eq: {corr_ch4_co2:.2f}")
    print(f"Correlación Pearson entre CH4_eq y N2O_eq: {corr_ch4_n2o:.2f}")
    print(f"Correlación Pearson entre CO2_eq y N2O_eq: {corr_co2_n2o:.2f}")
else:
    print("No hay suficientes datos para calcular la correlación.")